 ## Puzle de 8 con RL
 
Se aplica el algoritmo Q-learning para diseñar un agente que aprenda a resolver el puzzle del 8. 
 

Para poder aplicar Q-learning necesitamos numerar las diferentes acciones y estados para poder crear la q_table.

In [ ]:
import numpy as np
import random
import math
from IPython.display import clear_output

In [ ]:
from search import Problem
import itertools
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
        
    
    def actions(self,estado):
     
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        if pos_hueco not in (0,3,6): 
            accs.append("Mover hueco izquierda") 
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
                
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 3]
            l[pos_hueco + 3] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
    # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
    #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0
    
    def lista_estados(self,estado):
        i = 0
        diccionario = {}
        for e in itertools.permutations(estado):
            diccionario[e] = i
            i = i + 1
        return diccionario  
    
    def listaAcciones(self):
        listaAcciones = {}
        listaAcciones[0] ="Mover hueco arriba" 
        listaAcciones[1] ="Mover hueco abajo"
        listaAcciones[2] ="Mover hueco derecha"
        listaAcciones[3] ="Mover hueco izquierda"
        return listaAcciones
    
    def listaAcciones2(self):
        listaAcciones2 = {}
        listaAcciones2["Mover hueco arriba"] = 0
        listaAcciones2["Mover hueco abajo"] =1
        listaAcciones2["Mover hueco derecha"] =2
        listaAcciones2["Mover hueco izquierda"] =3
        return listaAcciones2

In [ ]:
# creamos e inicializamos la Q-Table
totalAcciones = 4
totalEstados = math.factorial(9)
q_table = np.zeros([totalEstados, totalAcciones])    

In [ ]:
initial=(1,2,0,4,5,3,7,8,6)

In [ ]:
ocho = Ocho_Puzzle(initial)
print("estado objetivo",ocho.goal)   
ocho.__init__(initial) 
state = ocho.initial


In [ ]:
def recompensa(state):
    if state==ocho.goal:
        return 100
    else:
        return -1

In [ ]:
def agente_puzle8(episodios=10000, alpha = 0.1, gamma = 0.6, epsilon = 0.8):
    """Creamos y entrenamos el agente con los parametros dados"""
    state = ocho.initial
    listaAcciones2 = ocho.listaAcciones2()
    listaAcciones = ocho.listaAcciones()
    listaEstados = ocho.lista_estados(state)
    exitos=0
    for i in range(1, episodios):
        if random.uniform(0, 1) < epsilon:
            mejor = listaAcciones2[random.choice(ocho.actions(state))]
            #print("La accion aleatoria seleccionada es ", listaAcciones[mejor])
        else:
            mejor = 0
            for accion in listaAcciones.keys():
                if q_table[listaEstados[state]][accion] > q_table[listaEstados[state]][mejor]:
                    mejor = accion
                    #print("La mejor accion es ", listaAcciones[mejor])

        if (listaAcciones[mejor] in ocho.actions(state)):
            sig_estado = ocho.result(state , listaAcciones[mejor])
            reward = recompensa(sig_estado)
        else:
            reward = -50 #accion no aplicable
            sig_estado =state
        old_value = q_table[listaEstados[state]][mejor]
        next_max = np.max(q_table[listaEstados[sig_estado]][mejor])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[listaEstados[state], mejor] = new_value
        state = sig_estado
        if state == ocho.goal:
            #Termino el episodio con éxito. Se ha encontrado la solución
            state = tuple(random.sample(ocho.goal, len(ocho.goal)))
            #state = ocho.initial
            exitos=exitos+1
            continue
        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")

    print("Training finished.\n") 
    print("La Q-table:\n") 
    print(q_table)
    # Métrica del % de veces que ha llegado al estado objetivo. 
    print("Número de éxitos: ", exitos)

In [ ]:
# MODIFICA LOS PARÁMETROS Y OBSERVA EL APRENDIZAJE
alpha = 0.1
gamma = 0.5
epsilon = 0.8
episodios=1000000

In [ ]:
# puedes medir el tiempo de aprendizaje 
#%%time
agente_puzle8(episodios,alpha,gamma,epsilon)

In [ ]:
def evalua_agente_puzle8(initial):
    """Solo explotación del agente sin modificar la tabla Q"""
    exitos=0
    fallos=0
    state = initial
    episodios =100
    #listaAcciones2 = ocho.listaAcciones2()
    listaAcciones = ocho.listaAcciones()
    listaEstados = ocho.lista_estados(state)
    for i in range(1, episodios+1):
        done =False
        while not done:
            mejor_accion = 0        
            for accion in listaAcciones.keys():
                if q_table[listaEstados[state]][accion] > q_table[listaEstados[state]][mejor_accion]:
                    mejor_accion = accion                
            print("La accion aleatoria seleccionada es ", listaAcciones[mejor_accion])
            print("Estado:", state)
            if (listaAcciones[mejor_accion] in ocho.actions(state)):
                state = ocho.result(state , listaAcciones[mejor_accion])
                print("Estado siguiente:", state)
            else: 
                #termino con fallo porque la accion no es aplicable
                fallos=fallos+1
                done = True            
            if state == ocho.goal:
                #print("Termino el episodio con éxito. Se ha encontrado la solución")
                exitos=exitos+1 
                done =True 
                state=initial
            if i % 100 == 0:
                clear_output(wait=True)
                print(f"Episodio: {i}")
        
    print("Evaluación terminada.\n") 
    # Métrica del % de veces que ha llegado al estado objetivo. 
    print("Número de éxitos:", exitos)
    print("Número de éxitos:", (exitos/episodios)*100, " %")
    
    

In [ ]:
evalua_agente_puzle8(initial)

El agente ha aprendido con QLearning a resolver el problema del 8 puzle tardando un cierto tiempo.
Una vez entrenado puedes ver qué tal se comporta (usando sólo la parte de explotación de la matriz Q aprendida y midiendo el porcentaje de éxitos). 

#### Responde a las siguientes cuestiones:

- Justifica de forma razonada si esta aproximación con aprendizaje es mejor que la aproximación con búsqueda heurística de la primera práctica.
- Observa la matriz de aprendizaje ¿qué ha aprendido el agente? __El agente aprende cuál es la mejor opción en cada estado.__
- Si entrenas el agente para un estado inicial ¿cómo se comporta al evaluarlo con otro estado inicial? __El número de éxito será muy baja (0) y si el estado inicial es similar al estado inicial origen, el tiempo de ejecución va muy larga.__
- Realiza los cambios necesarios para que el agente aprenda a resolver el problema desde cualquier estado inicial. __Cuando el trainning terminar con éxito, no va a volver al mismo estado inicial, va poner un estado cualquier con el codigo "state = tuple(random.sample(ocho.goal, len(ocho.goal)))"__
- Te parece adecuada la función de recompensa utilizada. ¿Crees que se podría mejorar el rendimiento del agente cambiando la recompensa? __Si, pongo -1 si el estado actual no es el objetivo de estado, para que pueda encontrar la solución con el menor número de operaciones.__
- Comprueba cómo se comporta el aprendizaje con distintas configuraciones de los parámetros. Indica claramente cuál es la mejor configuración que has encontrado. __alpha = 0.1, gamma = 0.5, epsilon = 0.8__